In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install glove-python-binary

     |████████████████████████████████| 948 kB 7.6 MB/s 


In [ ]:
from glove import Glove
glove=Glove()
glove_model=glove.load("/content/drive/MyDrive/FYP/CAA_NRC/Glove/glove.model")

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=pd.get_dummies(df["label"])
df

,label,new_text
0,0,muslim immigrant says that we will conquer ger...
1,1,my message to indian muslims kashmir is the sp...
2,2,normally never i abuse anyone on twitter but t...
3,1,heard someone suggesting holding anti caa nrc ...
4,2,the apex court has issued a notice to the unio...
...,...,...
3096,2,save hindus infiltrators anti national go to p...
3097,2,breaking bhimarmychief chandrashekhar azad has...
3098,0,tamil nadu cm edappadi k palaniswamy says citi...
3099,2,some more photos of protest meetings against c...


In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [ ]:
import tensorflow as tf
maxlen=66
max_words=len(glove_model.dictionary)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [ ]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in glove_model.dictionary:
    embed_vector=glove_model.word_vectors[glove_model.dictionary[word]]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [ ]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.10990729,  0.04230337,  0.05654864, ..., -0.20832376,
        -0.03223338, -0.21452594],
       [ 0.08457289, -0.06926741, -0.00822759, ..., -0.15099157,
        -0.02485671, -0.19643595],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense,Flatten
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=66,embeddings_initializer=Constant(embed_matrix)))
model.add(Flatten()) # loss stucks at about 
# model.add(Dense(32,activation='relu'))
# model.add(Dropout(0.60))
# model.add(Dense(16,activation='relu'))
# model.add(Dropout(0.60))
# model.add(Dense(1,activation='softmax'))
model = Sequential()
model.add(Dense(32, input_dim=X.shape[1],activation="sigmoid"))
model.add(Dropout(0.6))
# model.add(Dense(32,activation="sigmoid"))
# model.add(Dropout(0.6))
model.add(Dense(16,activation="sigmoid"))
model.add(Dropout(0.6))
model.add(Dense(3,activation="softmax"))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

Epoch 1/100
37/37 [==============================] - 3s 3ms/step - loss: 1.3945 - accuracy: 0.4103
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 1.2596 - accuracy: 0.4009
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 1.2097 - accuracy: 0.3927
Epoch 4/100
37/37 [==============================] - 0s 3ms/step - loss: 1.1720 - accuracy: 0.3862
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 1.1550 - accuracy: 0.4022
Epoch 6/100
37/37 [==============================] - 0s 3ms/step - loss: 1.1461 - accuracy: 0.3849
Epoch 7/100
37/37 [==============================] - 0s 3ms/step - loss: 1.1324 - accuracy: 0.3983
Epoch 8/100
37/37 [==============================] - 0s 2ms/step - loss: 1.1034 - accuracy: 0.3983
Epoch 9/100
37/37 [==============================] - 0s 3ms/step - loss: 1.1058 - accuracy: 0.3897
Epoch 10/100
37/37 [==============================] - 0s 3ms/step - loss: 1.1034 - accuracy: 0.3918
Epoch 11/

In [ ]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
# display_metrics(y_test,y_pred)

In [ ]:
import numpy as np
y_pred=model.predict(X_test)
y_test=y_test.to_numpy()
l=[np.argmax(i) for i in y_pred]
M=[np.argmax(i) for i in y_test]
display_metrics(M,l)

0.45489690721649484
0.20844405078240333
0.15163230240549827
0.3333333333333333


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
